In [1]:
import boto3
import time

In [3]:
import pandas as pd

In [2]:
import pandas as pd
from io import StringIO

In [6]:
import configparser

In [7]:
config = configparser.ConfigParser()

In [1]:
config.read_file(open('post.config'))

NameError: name 'config' is not defined

In [6]:
athenaClient = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION)

In [34]:
Dict = {}
def download_and_load_query_results(client, query_response):
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(2)
            else:
                raise err
                
    temp_file_location = "athena_query_results.csv"
    s3_client = boto3.client("s3",
                            aws_access_key_id=AWS_ACCESS_KEY,
                            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                            region_name=AWS_REGION)
    s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location)
    return pd.read_csv(temp_file_location)

In [37]:
response = athenaClient.start_query_execution(
    QueryString = "SELECT * FROM enigma_jhu;",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}    
    })

In [4]:
csv_buff = StringIO()

In [87]:
s3Reso.Object(bucket,'output/dimHospital.csv').put(Body=csv_buff.getvalue())

{'ResponseMetadata': {'RequestId': 'DR9WZPDT7AJSRNKK',
  'HostId': 'SmuwdPDosEwW0yMerj62GpmeBAf95ETk0TkiNqcs7dSOLc/m6BGMQI3GscyNPqmR0/Q6Rkvdma8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SmuwdPDosEwW0yMerj62GpmeBAf95ETk0TkiNqcs7dSOLc/m6BGMQI3GscyNPqmR0/Q6Rkvdma8=',
   'x-amz-request-id': 'DR9WZPDT7AJSRNKK',
   'date': 'Sun, 30 Oct 2022 09:53:49 GMT',
   'etag': '"052b91a93c128f0920be05c2066e5f13"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"052b91a93c128f0920be05c2066e5f13"'}

In [88]:
dimRegion2.to_csv(csv_buff)

In [47]:
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE=config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES=config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE=config.get('DWH','DWH_NODE_TYPE')
DWH_CLUSTER_IDENTIFIER=config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB=config.get('DWH','DWH_DB')
DWH_DB_USER=config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD=config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT=config.get('DWH','DWH_PORT')
DWH_IAM_ROLE_NAME=config.get('DWH','DWH_IAM_ROLE_NAME')

In [48]:
config.get('DWH','DWH_IAM_ROLE_NAME')

'redshiftRole'

In [17]:
ec2 = boto3.resource('ec2',
                    region_name="us-east-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

In [43]:
iam = boto3.client('iam',
                    region_name="us-east-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

In [21]:
redshift = boto3.client('redshift',
                    region_name="us-east-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

In [22]:
s3 = boto3.resource('s3',
                    region_name="us-east-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

In [33]:
bucket = s3.Bucket("de-glued")

In [36]:
locFile = [filename.key for filename in bucket.objects.filter(Prefix='output/Unsaved/2022/10')]

In [53]:
roleARN = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)

In [58]:
iamArn = roleARN['Role']['Arn']

In [59]:
try:
    response = redshift.create_cluster(
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        IamRoles=[iamArn]
    )
except Exception as e:
    print(e)

In [66]:
clusterProp = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [68]:
DWH_ENDPOINT = clusterProp['Endpoint']['Address']
DWH_ROLE_ARN = clusterProp['IamRoles'][0]['IamRoleArn']
DB_NAME=clusterProp['DBName']
DB_USER= clusterProp['MasterUsername']

In [69]:
DWH_ENDPOINT

'redshift-covid.cyg2sluuahwh.us-east-1.redshift.amazonaws.com'

In [70]:
clusterProp['VpcId']

'vpc-0b4db824c1d632c16'

In [74]:
try:
    vpc = ec2.Vpc(id=clusterProp['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp='0.0.0.0/0',
            IpProtocol='TCP',
            FromPort=int(DWH_PORT),
            ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0290e6034c427a02a')


In [75]:
import redshift_connector

In [76]:
import psycopg2

In [113]:
try:
    conn = psycopg2.connect(host=DWH_ENDPOINT,dbname=DB_NAME,user=DB_USER,password=DWH_DB_PASSWORD,port=DWH_PORT)
except Exception as e:
    print(e)

In [114]:
conn.set_session(autocommit=True)

In [115]:
try:
    cur = conn.cursor()
    
except:
    print(e)

In [133]:
try:
    cur.execute("""
        CREATE TABLE "factcorrected" (
            "index" INTEGER,
          "fips" REAL,
          "province_state" TEXT,
          "country_region" TEXT,
          "confirmed" REAL,
          "deaths" REAL,
          "recovered" REAL,
          "active" REAL,
          "date" INTEGER,
          "state" TEXT,
          "positive" REAL,
          "negative" REAL,
          "hospitalizedcurrently" REAL,
          "hospitalized" REAL,
          "hospitalizeddischarged" REAL
    )
    """)
except Exception as e:
    print(e)

In [125]:
DWH_ROLE_ARN

'arn:aws:iam::642924624251:role/redshiftRole'

In [134]:
try:
    cur.execute("""
        COPY factcorrected from 's3://de-glued/output/factcovid.csv'
        CREDENTIALS 'aws_iam_role=arn:aws:iam::642924624251:role/redshiftRole'
        DELIMITER ','
        region 'us-east-1'
    """)
except Exception as e:
    print(e)

Load into table 'factcorrected' failed.  Check 'stl_load_errors' system table for details.

